In [2]:
train_rf_df = spark.read.format("csv") \
                .option('header', True) \
                .option('inferSchema', True) \
                .load("s3://raghu18/trends/df_train4.csv")




In [3]:
x_test = spark.read.format("csv") \
                .option('header', True) \
                .option('inferSchema', True) \
                .load("s3://raghu18/trends/df_test.csv")

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier as RF
from pyspark.ml.regression import RandomForestRegressor as RF_reg
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
import functools
from pyspark.ml.feature import OneHotEncoder
#import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
from pyspark.ml.evaluation import RegressionEvaluator


In [5]:
# prepare labeled sets    Train
cols_now = ['user_total_orders', 'user_total_items', 'total_distinct_items',
       'user_average_days_between_orders', 'user_average_basket',
       'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio',
       'aisle_id', 'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'UP_orders', 'UP_orders_ratio',
       'UP_average_pos_in_cart', 'UP_reorder_rate', 'UP_orders_since_last',
       'UP_delta_hour_vs_last']
assembler_features = VectorAssembler(inputCols=cols_now, outputCol='features')
label_indexer = StringIndexer(inputCol='label', outputCol='label_double')
pipeline_train = Pipeline(stages=[assembler_features, label_indexer])

In [6]:
# prepare labeled sets     x_test
cols_now = ['user_total_orders', 'user_total_items', 'total_distinct_items',
       'user_average_days_between_orders', 'user_average_basket',
       'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio',
       'aisle_id', 'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'UP_orders', 'UP_orders_ratio',
       'UP_average_pos_in_cart', 'UP_reorder_rate', 'UP_orders_since_last',
       'UP_delta_hour_vs_last']
assembler_features = VectorAssembler(inputCols=cols_now, outputCol='features')
pipeline_test = Pipeline(stages=[assembler_features])

In [7]:
trainData = pipeline_train.fit(train_rf_df).transform(train_rf_df)

testData = pipeline_test.fit(x_test).transform(x_test)

In [12]:
#Grid search
rf = RF(labelCol='label_double', featuresCol='features')
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [1, 2]) \
    .addGrid(rf.maxDepth, [2]) \
    .addGrid(rf.maxBins, [2]) \
    .build()
pipeline = Pipeline([rf])

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=4)  # use 3+ folds in practice


Method __init__ forces keyword arguments.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/__init__.py", line 103, in wrapper
    raise TypeError("Method %s forces keyword arguments." % func.__name__)
TypeError: Method __init__ forces keyword arguments.



In [ ]:
cvModel = crossval.fit(training)

In [13]:
rf = RF_reg(labelCol='label_double', featuresCol='features',numTrees=30, maxDepth=10, maxBins=100)
fit = rf.fit(trainData)
# transformed = fit.transform(testData)

In [14]:
transformed = fit.transform(testData)

In [15]:
testData.show(2)

+---+--------+----------+-----------------+----------------+--------------------+--------------------------------+-------------------+-----------------+----------------------+----------------+--------+-------------+--------------+----------------+--------------------+---------+---------------+----------------------+---------------+--------------------+---------------------+------------------+--------------------+
|_c0|order_id|product_id|user_total_orders|user_total_items|total_distinct_items|user_average_days_between_orders|user_average_basket|order_hour_of_day|days_since_prior_order|days_since_ratio|aisle_id|department_id|product_orders|product_reorders|product_reorder_rate|UP_orders|UP_orders_ratio|UP_average_pos_in_cart|UP_reorder_rate|UP_orders_since_last|UP_delta_hour_vs_last|              pred|            features|
+---+--------+----------+-----------------+----------------+--------------------+--------------------------------+-------------------+-----------------+--------------

In [17]:
result = transformed.select(['order_id', 'product_id', 'pred'])

In [18]:
from io import StringIO
import boto3

csv_buffer = StringIO()
result.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df.csv').put(Body=csv_buffer.getvalue())

No module named boto3
Traceback (most recent call last):
ImportError: No module named boto3



In [20]:
result.write.save("s3://raghu18/trends/file.csv", format='csv', header=True)